In [221]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
csv_path = "benchmark_new.csv"
df = pd.read_csv(csv_path)

df['batch_size'] = df['batch_size'].astype(int)
df['wall_time_ms'] = df['wall_time_ms'].astype(float)
df["top_k"] = pd.to_numeric(df["top_k"], errors="coerce")
df["cpu_nthreads"] = pd.to_numeric(df["cpu_nthreads"], errors="coerce")
display(df.head())
display(df.info())

sns.set_theme(style="whitegrid")

In [ ]:
threshold_default_csv_path = "benchmark_numa.csv"
threshold_default_df = pd.read_csv(threshold_default_csv_path)

index_name = "Flat"
threshold_default_df = threshold_default_df[
    (threshold_default_df["index_name"] == index_name)
    & (
        (threshold_default_df["top_k"] == 1)
        | (threshold_default_df["top_k"] == 4)
    )
]

g = sns.FacetGrid(threshold_default_df, col="top_k", height=4)
g.map_dataframe(sns.lineplot, x="batch_size",
                y="wall_time_ms", hue="hw", marker="o")
g.add_legend(title="cpu")
g.set_xticklabels(rotation=45, ha="right")
g.set_axis_labels("Batch Size", "Wall Time (ms)")
g.set_titles("top-k = {col_name}")
g.set(xbound=(0, 128))
g.fig.suptitle(f"Flat Idx CPU (BLAS threshold=20)", y=1.05)

In [224]:
size_map_bytes = {
    "Flat": 3072000045,
    "HNSW32_Flat": 3344129506,
    "HNSW32_PQ16": 288915979,
    "IVF1024_Flat": 3083154059,
    "IVF1024_PQ16": 27940532,
    "IVF4096_Flat": 3092615819,
    "IVF4096_PQ16": 37402292,
    "IVF65536_Flat": 3281851019,
    "IVF65536_HNSW32_Flat": 3299688512,
    "IVF65536_HNSW32_PQ16": 244474985,
    "IVF65536_PQ16": 226637492,
    "OPQ16_64_Flat": 256196724,
    "OPQ16_64_HNSW32_Flat": 528326185,
    "OPQ16_64_HNSW32_PQ16": 288391762,
    "OPQ16_64_IVF1024_Flat": 264467154,
    "OPQ16_64_IVF1024_PQ16": 24532731,
    "OPQ16_64_IVF4096_Flat": 265278162,
    "OPQ16_64_IVF4096_PQ16": 25343739,
    "OPQ16_64_IVF65536_Flat": 281498322,
    "OPQ16_64_IVF65536_HNSW32_Flat": 299335815,
    "OPQ16_64_IVF65536_HNSW32_PQ16": 59401392,
    "OPQ16_64_IVF65536_PQ16": 41563899,
    "OPQ16_64_PQ16": 16262301,
    "PCA64_Flat": 258562457,
    "PCA64_HNSW32_Flat": 530691918,
    "PCA64_HNSW32_PQ16": 290757495,
    "PCA64_IVF1024_Flat": 266832887,
    "PCA64_IVF1024_PQ16": 26898464,
    "PCA64_IVF4096_Flat": 267643895,
    "PCA64_IVF4096_PQ16": 27709472,
    "PCA64_IVF65536_Flat": 283864055,
    "PCA64_IVF65536_HNSW32_Flat": 301701548,
    "PCA64_IVF65536_HNSW32_PQ16": 61767125,
    "PCA64_IVF65536_PQ16": 43929632,
    "PCA64_PQ16": 18628034,
    "PQ16": 16786518,
}
df['size_b'] = df['index_name'].map(size_map_bytes)

In [225]:
cond_pretransform = df["index_name"].str.contains("OPQ|PCA", na=False)
cond_hnsw = df["index_name"].str.contains(
    "HNSW", na=False) & ~df["index_name"].str.contains("IVF", na=False)
cond_ivf = df["index_name"].str.contains("IVF", na=False)

df["category"] = np.where(
    cond_hnsw,
    "HNSW",
    np.where(cond_ivf, "IVF", "Exhaustive")
)

In [ ]:
# Flat Index CPU
index_name = "Flat"
df_cpu = df[
    (df["index_name"] == index_name)
    & (
        (df["top_k"] == 1)
        | (df["top_k"] == 4)
    )
]

g = sns.FacetGrid(df_cpu, col="top_k", height=4)
g.map_dataframe(sns.lineplot, x="batch_size",
                y="wall_time_ms", hue="hw", marker="o")
g.add_legend(title="cpu")
g.set_xticklabels(rotation=45, ha="right")
g.set_axis_labels("Batch Size", "Wall Time (ms)")
g.set_titles("top-k = {col_name}")
g.set(xbound=(0, 128))
g.fig.suptitle(f"Flat Idx CPU (BLAS threshold=1)", y=1.05)

In [ ]:
# Index perf. w/o quantization
quantization_keys = ["PQ16", "OPQ", "PCA"]
quant_pattern = "|".join(quantization_keys)
df_wo_quant = df[~df["index_name"].str.contains(
    quant_pattern, case=False, na=False)]
df_wo_quant = df_wo_quant[
    ~(df_wo_quant['index_name'].str.contains("HNSW", na=False) & df_wo_quant["index_name"].str.contains("IVF", na=False))
    & ((df_wo_quant["top_k"] == 2))
    & ((df_wo_quant["nprobe"] == 4) | (df_wo_quant["nprobe"].isna()) | (df_wo_quant["nprobe"] == 0))
    & ((df_wo_quant["efSearch"] == 16) | (df_wo_quant["efSearch"].isna()) | (df_wo_quant["efSearch"] == 0))
]
g = sns.FacetGrid(df_wo_quant, col="top_k", row="hw")
g.map_dataframe(sns.lineplot, x="batch_size", y="wall_time_ms",
                hue="index_name",  marker="o")
g.add_legend(title="Index")
g.set_axis_labels("Batch Size", "Wall Time (ms)")
g.set_titles("top-k = {col_name}, hw={row_name}")
g.set(yscale="log")
g.fig.suptitle(f"Index perf. w/o quantization", y=1.05)

In [ ]:
# Non-Quantized Index Size & Avg. Latency
df_no_quant = df[~df["index_name"].str.contains(
    quant_pattern, case=False, na=False)]
g = sns.FacetGrid(df_no_quant, row="hw",
                  hue="index_name", height=4, aspect=1.2)
g.map_dataframe(sns.scatterplot, x="size_b", y="wall_time_ms",  legend=False)
g.add_legend(title="Index", prop={"size": 12},
             title_fontsize=14, )
g.set_axis_labels("Index Size(B)", "Wall Time (ms)")
g.set_titles("HW = {row_name}")
g.fig.suptitle(f"Index Size & Avg. Latency", y=1.05)

In [ ]:
# Quantized Index Size & Avg. Latency
df_no_quant = df[df["index_name"].str.contains(
    quant_pattern, case=False, na=False)]
df_no_quant = df_no_quant[
    (~df_no_quant['index_name'].str.contains("IVF"))
    & (~df_no_quant['index_name'].str.contains("HNSW", na=False))
]
g = sns.FacetGrid(df_no_quant, row="hw",
                  hue="index_name", height=4, aspect=1.2)
g.map_dataframe(sns.scatterplot, x="size_b", y="wall_time_ms",  legend=False)
g.add_legend(title="Index", prop={"size": 14},
             title_fontsize=14, )
g.set_axis_labels("Index Size(B)", "Wall Time (ms)")
g.set_titles("HW = {row_name}")
g.fig.suptitle(f"Quantized Index", y=1.05)

In [ ]:
# IVF nprobe perf. w/o quantization
df_ivf = df[
    (df["index_name"].str.contains("IVF", na=False))
    & (~df["index_name"].str.contains(quant_pattern, case=False, na=False))
    & (df['batch_size'] == 4)
]

g = sns.FacetGrid(
    df_ivf,
    col="batch_size",
    row="hw",
    hue="index_name",
    sharey=False,
    height=4,
    aspect=1.3,
)

g.map_dataframe(sns.lineplot, x="nprobe",
                y="wall_time_ms", marker="o", legend=False)

g.add_legend(title="Index Name")
g.set_axis_labels("nprobe", "Wall Time (ms)")
g.set_titles("batch = {col_name}, hw = {row_name}")
g.fig.suptitle("IVF latency w.r.t. nprobe", y=1.05)
plt.tight_layout()
plt.show()

In [ ]:
# HNSW efSearch perf. w/o quantization
df_hnsw = df[
    (df["index_name"].str.contains("HNSW", na=False))
    & (~df["index_name"].str.contains("IVF", case=False, na=False))
    & (~df["index_name"].str.contains(quant_pattern, case=False, na=False))
    & (
        (df['top_k'] == 2) &
        (df['batch_size'] == 4)
    )
]

g = sns.FacetGrid(
    df_hnsw,
    col="batch_size",
    row="top_k",
    height=4,
    aspect=1.3
)

g.map_dataframe(sns.lineplot, x="efSearch", y="wall_time_ms",
                hue="hw", style="index_name", marker="o")

g.add_legend(title="Index")
g.set_axis_labels("efSearch", "Wall Time (ms)")
g.set_titles("batch = {col_name}, top-k = {row_name}")
g.fig.suptitle("HNSW latency wrt. efSearch", y=1.05)
plt.tight_layout()
plt.show()

In [ ]:
# Flat Index perf. w/ Quantization
df_quant = df[
    ~(df["index_name"].str.contains("HNSW", case=False, na=False))
    & ~(df["index_name"].str.contains("IVF", na=False))
    & (
        (df['top_k'] == 2)
    )
]

g = sns.FacetGrid(
    df_quant,
    col="top_k",
    row="hw",
    hue="index_name",
    height=4,
    aspect=1.3,
)
g.map_dataframe(sns.lineplot, x="batch_size",
                y="wall_time_ms", marker="o", legend=False)
g.add_legend(title="Index",
             prop={"size": 14},
             title_fontsize=14, )
g.set_axis_labels("Batch Size", "Wall Time (ms) log-scale")
g.set_titles("top_k = {col_name}, hw = {row_name}")
g.set(yscale="log")
g.fig.suptitle("Flat Index w/ Quantization", y=1.05)

In [ ]:
# GPU Support Index perf. w/ Quantization
pre_pattern = "|".join(["OPQ", "PCA"])
df_gpu_support = df[
    (
        # (df["index_name"].str.contains("HNSW", case=False, na=False) &~(df["index_name"].str.contains("IVF", na=False)))
        ((df["index_name"].str.contains("IVF", na=False)) & ~(df["index_name"].str.contains("HNSW", na=False) & df["index_name"].str.contains("IVF", na=False))
         | ~(((df["index_name"].str.contains("HNSW", case=False, na=False)) | (df["index_name"].str.contains("IVF", na=False))) | (df["index_name"].str.contains(quant_pattern, na=False)))
         )
    )
    & ~(df["index_name"].str.contains(pre_pattern, case=False, na=False))
    & (
        (df['top_k'] == 2)
        & ((df["nprobe"] == 4) | (df["nprobe"].isna()) | (df["nprobe"] == 0))
        & ((df["efSearch"] == 16) | (df["efSearch"].isna()) | (df["efSearch"] == 0))
    )
]

g = sns.FacetGrid(
    df_gpu_support,
    col="top_k",
    row="hw",
    hue="index_name",
    height=4,
    aspect=1.3,
)
g.map_dataframe(sns.lineplot, x="batch_size",
                y="wall_time_ms", marker="o", legend=False)
g.add_legend(title="Index",
             prop={"size": 14},
             title_fontsize=14, )
g.set_axis_labels("Batch Size", "Wall Time (ms)")
g.set_titles("top_k = {col_name}, hw = {row_name}")